In [17]:
!pip install medpy

In [18]:


import numpy as np
from keras.models import Model, load_model
from keras.layers import BatchNormalization,Convolution2D,SeparableConv2D,Conv2D,concatenate,add,Concatenate,Conv2DTranspose, Input, Add, UpSampling2D, Activation, merge, MaxPooling2D, Deconvolution2D, Reshape, Permute, Dropout
from keras.optimizers import SGD, Adam, Adadelta,RMSprop
#from scipy.misc import imresize, imsave, imread
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau
from medpy.metric import dc, precision, recall
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

import numpy as np
from keras.models import Model, load_model
from keras.layers import BatchNormalization,Convolution2D, Input,ZeroPadding2D, UpSampling2D, Activation, merge, MaxPooling2D, Deconvolution2D, Reshape, Permute

from keras.optimizers import SGD, Adam
#from scipy.misc import imresize, imsave, imread
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from medpy.metric import dc, precision, recall
import matplotlib.pyplot as plt
from keras.regularizers import l2
from sklearn.metrics import log_loss
from keras.preprocessing.image import ImageDataGenerator
import sys
lrate1=3e-4  #changed lrate
lrate=float(lrate1)
print ('learning rate:',lrate)
#print('JBHI_w8_'+str(lrate1)+'.h5')
#print('JBHI_model_best'+lrate1)

import numpy as np
import warnings

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs

import re
from scipy import linalg
import scipy.ndimage as ndi
# from six.moves import range
import os
import threading
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter




learning rate: 0.0003


In [0]:
def random_channel_shift(x, intensity, channel_index=0):
    x = np.rollaxis(x, channel_index, 0)
    min_x, max_x = np.min(x), np.max(x)
    channel_images = [np.clip(x_channel + np.random.uniform(-intensity, intensity), min_x, max_x)
                      for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_index+1)
    return x


def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix


def apply_transform(x, transform_matrix, channel_index=0, fill_mode='nearest', cval=0.):
    x = np.rollaxis(x, channel_index, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(x_channel, final_affine_matrix,
                      final_offset, order=0, mode=fill_mode, cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_index+1)
    return x


def flip_axis(x, axis):
    x = np.asarray(x).swapaxes(axis, 0)
    x = x[::-1, ...]
    x = x.swapaxes(0, axis)
    return x


def array_to_img(x, dim_ordering='default', scale=True):
    from PIL import Image
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    if dim_ordering == 'th':
        x = x.transpose(1, 2, 0)
    if scale:
        x += max(-np.min(x), 0)
        x /= np.max(x)
        x *= 255
    if x.shape[2] == 3:
        # RGB
        return Image.fromarray(x.astype('uint8'), 'RGB')
    elif x.shape[2] == 1:
        # grayscale
        return Image.fromarray(x[:, :, 0].astype('uint8'), 'L')
    else:
        raise Exception('Unsupported channel number: ', x.shape[2])


def img_to_array(img, dim_ordering='default'):
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    if dim_ordering not in ['th', 'tf']:
        raise Exception('Unknown dim_ordering: ', dim_ordering)
    # image has dim_ordering (height, width, channel)
    x = np.asarray(img, dtype='float32')
    if len(x.shape) == 3:
        if dim_ordering == 'th':
            x = x.transpose(2, 0, 1)
    elif len(x.shape) == 2:
        if dim_ordering == 'th':
            x = x.reshape((1, x.shape[0], x.shape[1]))
        else:
            x = x.reshape((x.shape[0], x.shape[1], 1))
    else:
        raise Exception('Unsupported image shape: ', x.shape)
    return x


class ImageDataGenerator(object):
    '''Generate minibatches with
    real-time data augmentation.
    Assume X is train img, Y is train label (same size as X with only 0 and 255 for values)
    # Arguments
        featurewise_center: set input mean to 0 over the dataset. Only to X
        samplewise_center: set each sample mean to 0. Only to X
        featurewise_std_normalization: divide inputs by std of the dataset. Only to X
        samplewise_std_normalization: divide each input by its std. Only to X
        zca_whitening: apply ZCA whitening. Only to X
        rotation_range: degrees (0 to 180). To X and Y
        width_shift_range: fraction of total width. To X and Y
        height_shift_range: fraction of total height. To X and Y
        shear_range: shear intensity (shear angle in radians). To X and Y
        zoom_range: amount of zoom. if scalar z, zoom will be randomly picked
            in the range [1-z, 1+z]. A sequence of two can be passed instead
            to select this range. To X and Y
        channel_shift_range: shift range for each channels. Only to X
        fill_mode: points outside the boundaries are filled according to the
            given mode ('constant', 'nearest', 'reflect' or 'wrap'). Default
            is 'nearest'. For Y, always fill with constant 0
        cval: value used for points outside the boundaries when fill_mode is
            'constant'. Default is 0.
        horizontal_flip: whether to randomly flip images horizontally. To X and Y
        vertical_flip: whether to randomly flip images vertically. To X and Y
        rescale: rescaling factor. If None or 0, no rescaling is applied,
            otherwise we multiply the data by the value provided (before applying
            any other transformation). Only to X
        dim_ordering: 'th' or 'tf'. In 'th' mode, the channels dimension
            (the depth) is at index 1, in 'tf' mode it is at index 3.
            It defaults to the `image_dim_ordering` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "th".
    '''
    def __init__(self,
                 featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=0.,
                 width_shift_range=0.,
                 height_shift_range=0.,
                 shear_range=0.,
                 zoom_range=0.,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=False,
                 vertical_flip=False,
                 rescale=None,
                 
                 dim_ordering='default'):
        
        if dim_ordering == 'default':
            dim_ordering = K.image_dim_ordering()
        self.__dict__.update(locals())
        self.mean = None
        self.std = None
        self.principal_components = None
        self.rescale = rescale

        if dim_ordering not in {'tf', 'th'}:
            raise Exception('dim_ordering should be "tf" (channel after row and '
                            'column) or "th" (channel before row and column). '
                            'Received arg: ', dim_ordering)
        self.dim_ordering = dim_ordering
        if dim_ordering == 'th':
            self.channel_index = 1
            self.row_index = 2
            self.col_index = 3
        if dim_ordering == 'tf':
            self.channel_index = 3
            self.row_index = 1
            self.col_index = 2

        if np.isscalar(zoom_range):
            self.zoom_range = [1 - zoom_range, 1 + zoom_range]
        elif len(zoom_range) == 2:
            self.zoom_range = [zoom_range[0], zoom_range[1]]
        else:
            raise Exception('zoom_range should be a float or '
                            'a tuple or list of two floats. '
                            'Received arg: ', zoom_range)

    def flow(self, X, y=None, batch_size=32, shuffle=True, seed=None,
             save_to_dir=None, save_prefix='', save_format='jpeg'):
        return NumpyArrayIterator(
            X, y, self,
            batch_size=batch_size, shuffle=shuffle, seed=seed,
            dim_ordering=self.dim_ordering,
            save_to_dir=save_to_dir, save_prefix=save_prefix, save_format=save_format)

    def standardize(self, x):
        # Only applied to X
        if self.rescale:
            x *= self.rescale
        # x is a single image, so it doesn't have image number at index 0
        img_channel_index = self.channel_index - 1
        if self.samplewise_center:
            x -= np.mean(x, axis=img_channel_index, keepdims=True)
        if self.samplewise_std_normalization:
            x /= (np.std(x, axis=img_channel_index, keepdims=True) + 1e-7)

        if self.featurewise_center:
            x -= self.mean
        if self.featurewise_std_normalization:
            x /= (self.std + 1e-7)

        if self.zca_whitening:
            flatx = np.reshape(x, (x.size))
            whitex = np.dot(flatx, self.principal_components)
            x = np.reshape(whitex, (x.shape[0], x.shape[1], x.shape[2]))

        return x

    def random_transform(self, x, y):
        # Need to modify to transform both X and Y ---- to do
        # x is a single image, so it doesn't have image number at index 0
        img_row_index = self.row_index - 1
        img_col_index = self.col_index - 1
        img_channel_index = self.channel_index - 1

        # use composition of homographies to generate final transform that needs to be applied
        if self.rotation_range:
            theta = np.pi / 180 * np.random.uniform(-self.rotation_range, self.rotation_range)
        else:
            theta = 0
        rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                    [np.sin(theta), np.cos(theta), 0],
                                    [0, 0, 1]])
        if self.height_shift_range:
            tx = np.random.uniform(-self.height_shift_range, self.height_shift_range) * x.shape[img_row_index]
        else:
            tx = 0

        if self.width_shift_range:
            ty = np.random.uniform(-self.width_shift_range, self.width_shift_range) * x.shape[img_col_index]
        else:
            ty = 0

        translation_matrix = np.array([[1, 0, tx],
                                       [0, 1, ty],
                                       [0, 0, 1]])
        if self.shear_range:
            shear = np.random.uniform(-self.shear_range, self.shear_range)
        else:
            shear = 0
        shear_matrix = np.array([[1, -np.sin(shear), 0],
                                 [0, np.cos(shear), 0],
                                 [0, 0, 1]])

        if self.zoom_range[0] == 1 and self.zoom_range[1] == 1:
            zx, zy = 1, 1
        else:
            zx, zy = np.random.uniform(self.zoom_range[0], self.zoom_range[1], 2)
        zoom_matrix = np.array([[zx, 0, 0],
                                [0, zy, 0],
                                [0, 0, 1]])

        transform_matrix = np.dot(np.dot(np.dot(rotation_matrix, translation_matrix), shear_matrix), zoom_matrix)

        h, w = x.shape[img_row_index], x.shape[img_col_index]
        transform_matrix = transform_matrix_offset_center(transform_matrix, h, w)
        x = apply_transform(x, transform_matrix, img_channel_index,
                            fill_mode=self.fill_mode, cval=self.cval)
        # For y, mask data, fill mode constant, cval = 0
        y = apply_transform(y, transform_matrix, img_channel_index,
                            fill_mode="constant", cval=0)

        if self.channel_shift_range != 0:
            x = random_channel_shift(x, self.channel_shift_range, img_channel_index)

        if self.horizontal_flip:
            if np.random.random() < 0.5:
                x = flip_axis(x, img_col_index)
                y = flip_axis(y, img_col_index)

        if self.vertical_flip:
            if np.random.random() < 0.5:
                x = flip_axis(x, img_row_index)
                y = flip_axis(y, img_row_index)

        # TODO:
        # channel-wise normalization
        # barrel/fisheye
        return x, y

    def fit(self, X,
            augment=False,
            rounds=1,
            seed=None):
        '''Required for featurewise_center, featurewise_std_normalization
        and zca_whitening.
        # Arguments
            X: Numpy array, the data to fit on.
            augment: whether to fit on randomly augmented samples
            rounds: if `augment`,
                how many augmentation passes to do over the data
            seed: random seed.
        # Only applied to X
        '''
        X = np.copy(X)
        if augment:
            aX = np.zeros(tuple([rounds * X.shape[0]] + list(X.shape)[1:]))
            for r in range(rounds):
                for i in range(X.shape[0]):
                    aX[i + r * X.shape[0]] = self.random_transform(X[i])
            X = aX

        if self.featurewise_center:
            self.mean = np.mean(X, axis=0)
            X -= self.mean

        if self.featurewise_std_normalization:
            self.std = np.std(X, axis=0)
            X /= (self.std + 1e-7)

        if self.zca_whitening:
            flatX = np.reshape(X, (X.shape[0], X.shape[1] * X.shape[2] * X.shape[3]))
            sigma = np.dot(flatX.T, flatX) / flatX.shape[1]
            U, S, V = linalg.svd(sigma)
            self.principal_components = np.dot(np.dot(U, np.diag(1. / np.sqrt(S + 10e-7))), U.T)


class Iterator(object):

    def __init__(self, N, batch_size, shuffle, seed):
        self.N = N
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.batch_index = 0
        self.total_batches_seen = 0
        self.lock = threading.Lock()
        self.index_generator = self._flow_index(N, batch_size, shuffle, seed)

    def reset(self):
        self.batch_index = 0

    def _flow_index(self, N, batch_size=32, shuffle=False, seed=None):
        # ensure self.batch_index is 0
        self.reset()
        while 1:
            if self.batch_index == 0:
                index_array = np.arange(N)
                if shuffle:
                    if seed is not None:
                        np.random.seed(seed + self.total_batches_seen)
                    index_array = np.random.permutation(N)

            current_index = (self.batch_index * batch_size) % N
            if N >= current_index + batch_size:
                current_batch_size = batch_size
                self.batch_index += 1
            else:
                current_batch_size = N - current_index
                self.batch_index = 0
            self.total_batches_seen += 1
            yield (index_array[current_index: current_index + current_batch_size],
                   current_index, current_batch_size)

    def __iter__(self):
        # needed if we want to do something like:
        # for x, y in data_gen.flow(...):
        return self

    def __next__(self, *args, **kwargs):
        # ?
        return self.next(*args, **kwargs)


class NumpyArrayIterator(Iterator):

    def __init__(self, X, y, image_data_generator,
                 batch_size=32, shuffle=False, seed=None,
                 dim_ordering='default',
                 save_to_dir=None, save_prefix='', save_format='jpeg'):
        if len(X) != len(y):
            raise Exception('X (images tensor) and y (labels) '
                            'should have the same length. '
                            'Found: X.shape = %s, y.shape = %s' % (np.asarray(X).shape, np.asarray(y).shape))
        if dim_ordering == 'default':
            dim_ordering = K.image_dim_ordering()
        self.X = X
        self.y = y
        self.image_data_generator = image_data_generator
        self.dim_ordering = dim_ordering
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        super(NumpyArrayIterator, self).__init__(X.shape[0], batch_size, shuffle, seed)

    def next(self):
        # for python 2.x.
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch
        # see http://anandology.com/blog/using-iterators-and-generators/
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock so it can be done in parallel
        batch_x = np.zeros(tuple([current_batch_size] + list(self.X.shape)[1:]))
        batch_y = np.zeros(tuple([current_batch_size] + list(self.y.shape)[1:]))
        for i, j in enumerate(index_array):
            x = self.X[j]
            label = self.y[j]
            x, label = self.image_data_generator.random_transform(x.astype('float32'), label.astype("float32"))
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x
            batch_y[i] = label
        if self.save_to_dir:
            for i in range(current_batch_size):
                img = array_to_img(batch_x[i], self.dim_ordering, scale=True)
                fname = '{prefix}_{index}_{hash}.{format}'.format(prefix=self.save_prefix,
                                                                  index=current_index + i,
                                                                  hash=np.random.randint(1e4),
                                                                  format=self.save_format)
                img.save(os.path.join(self.save_to_dir, fname))
                mask = array_to_img(batch_y[i], self.dim_ordering, scale=True)
                fname = '{prefix}_{index}_{hash}_mask.{format}'.format(prefix=self.save_prefix,
                                                                  index=current_index + i,
                                                                  hash=np.random.randint(1e4),
                                                                  format=self.save_format)
                mask.save(os.path.join(self.save_to_dir, fname))
        return batch_x, batch_y




In [0]:
def dice_coef(y_true, y_pred):
    
    #y_pred = K.round(y_pred)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return 1. -dice_coef(y_true, y_pred)


In [22]:
#TEST1_DATA

# 1w88Hj79XAbDr-nbQM5cWjLbqCi4sjRZW    --cutgrader1
# 1Y4XcQrdQ8qthROnMJ2-Krj7LUO0XWEAm    --cutgrader2
# 1DjNwEWftZtn_o0SUqRupGeR0eIk3S7qe    --cutgrader3


file7 = drive.CreateFile({'id': '1dhJjIXM_W4fm0U72h48Jfy2y3Eogl8ZJ'})
file7.GetContentFile('test1_cutimages_conall.npy')
test1_X = np.load('test1_cutimages_conall.npy')
print (test1_X.shape)

file8 = drive.CreateFile({'id': '1DjNwEWftZtn_o0SUqRupGeR0eIk3S7qe '})
file8.GetContentFile('test1_cutgrader3.npy')
test1_y = np.load('test1_cutgrader3.npy')
print (test1_y.shape)








(620, 256, 512)


ApiRequestError: ignored

In [23]:
#########

# test1_cutimages_conall     1dhJjIXM_W4fm0U72h48Jfy2y3Eogl8ZJ
# test1_cutgt1               1w88Hj79XAbDr-nbQM5cWjLbqCi4sjRZW
# test1_cutgt2               1Y4XcQrdQ8qthROnMJ2-Krj7LUO0XWEAm
# test1_cutgt3               1DjNwEWftZtn_o0SUqRupGeR0eIk3S7qe


#TEST1_DATA



file7 = drive.CreateFile({'id': '1dhJjIXM_W4fm0U72h48Jfy2y3Eogl8ZJ'})
file7.GetContentFile('test1_cutimages_conall.npy')
test1_X = np.load('test1_cutimages_conall.npy')
print (test1_X.shape)

#test1_cutgrader1
file8 = drive.CreateFile({'id': '1w88Hj79XAbDr-nbQM5cWjLbqCi4sjRZW'})
file8.GetContentFile('test1_cutgrader1.npy')
test1_y1= np.load('test1_cutgrader1.npy')
print (test1_y1.shape)


#test1_cutgrader2

file9 = drive.CreateFile({'id': '1Y4XcQrdQ8qthROnMJ2-Krj7LUO0XWEAm'})
file9.GetContentFile('test1_cutgrader2.npy')
test1_y2= np.load('test1_cutgrader2.npy')
print (test1_y2.shape)


#test1_cutgrader3


file10 = drive.CreateFile({'id': '1DjNwEWftZtn_o0SUqRupGeR0eIk3S7qe'})
file10.GetContentFile('test1_cutgrader3.npy')
test1_y3= np.load('test1_cutgrader3.npy')
print (test1_y3.shape)





(620, 256, 512)
(620, 256, 512)
(620, 256, 512)
(620, 256, 512)


In [24]:
#TEST2_DATA

#  1B_xojbfI3xpjWw_Wqpr89cIVkEp4aF2y  --cutgrader1
#  1dNvcUtc9G7TafH7ZQ0_lTkLANf1Va1be  --cutgrader2
#  1roXTQkkH46hWo6AU1UWJe4J3zhHSqfNr  --cutgrader3



file7 = drive.CreateFile({'id': '1vYOErUtk1aT7XnV9EcFXa1wI0b5twRE2'})
file7.GetContentFile('test2_cutimages_conall.npy')
test2_X = np.load('test2_cutimages_conall.npy')
print (test2_X.shape)

file8 = drive.CreateFile({'id': '1roXTQkkH46hWo6AU1UWJe4J3zhHSqfNr'})
file8.GetContentFile('test2_cutgrader1.npy')
test2_y = np.load('test2_cutgrader1.npy')
print (test2_y.shape)


##############

# test2_cutimage_conall   1vYOErUtk1aT7XnV9EcFXa1wI0b5twRE2
# test2_cutgt1            1B_xojbfI3xpjWw_Wqpr89cIVkEp4aF2y
# test2_cutgt2            1dNvcUtc9G7TafH7ZQ0_lTkLANf1Va1be
# test2_cutgt3            1roXTQkkH46hWo6AU1UWJe4J3zhHSqfNr



#TEST2_DATA



file1 = drive.CreateFile({'id': '1vYOErUtk1aT7XnV9EcFXa1wI0b5twRE2'})
file1.GetContentFile('test2_cutimages_conall.npy')
test2_X = np.load('test2_cutimages_conall.npy')
print (test2_X.shape)

#Test2_cutgrader1

file2 = drive.CreateFile({'id': '1B_xojbfI3xpjWw_Wqpr89cIVkEp4aF2y'})
file2.GetContentFile('test2_cutgrader1.npy')
test2_y1 = np.load('test2_cutgrader1.npy')
print (test2_y1.shape)

#test2_cutgrader2

file3 = drive.CreateFile({'id': '1dNvcUtc9G7TafH7ZQ0_lTkLANf1Va1be'})
file3.GetContentFile('test2_cutgrader2.npy')
test2_y2 = np.load('test2_cutgrader2.npy')
print (test2_y2.shape)


#test3_cutgrader3


file4 = drive.CreateFile({'id': '1roXTQkkH46hWo6AU1UWJe4J3zhHSqfNr'})
file4.GetContentFile('test2_cutgrader3.npy')
test2_y3 = np.load('test2_cutgrader3.npy')
print (test2_y3.shape)


     






(289, 256, 512)
(289, 256, 512)
(289, 256, 512)
(289, 256, 512)
(289, 256, 512)
(289, 256, 512)


In [25]:
train_X = np.reshape(train_X, (-1,256,512,1))
train_y = np.reshape(train_y*1, (-1,256,512,1))
test1_X = np.reshape(test1_X, (-1,256,512,1))
#test1_y = np.reshape(test1_y*1, (-1,256,512,1))


###########


test1_y1 = np.reshape(test1_y1*1, (-1,256,512,1))
test1_y2 = np.reshape(test1_y2*1, (-1,256,512,1))
test1_y3 = np.reshape(test1_y3*1, (-1,256,512,1))


test1_y1 = test1_y1.astype('float32')
test1_y2 = test1_y2.astype('float32')
test1_y3 = test1_y3.astype('float32')


###########




test2_X = np.reshape(test2_X, (-1,256,512,1))
test2_y = np.reshape(test2_y*1, (-1,256,512,1))


###########33


test2_y1 = np.reshape(test2_y1*1, (-1,256,512,1))
test2_y2 = np.reshape(test2_y2*1, (-1,256,512,1))
test2_y3 = np.reshape(test2_y3*1, (-1,256,512,1))


test2_y1 = test2_y1.astype('float32')
test2_y2 = test2_y2.astype('float32')
test2_y3 = test2_y3.astype('float32')


print(test2_X.shape)
print(test2_y1.shape)
print(test2_y2.shape)
print(test2_y3.shape)
############3

train_X = train_X.astype('float32')
train_y = train_y.astype('float32')
test1_X = test1_X.astype('float32')
#test1_y = test1_y.astype('float32')
test2_X = test2_X.astype('float32')
test2_y = test2_y.astype('float32')

print(train_X.shape)
print(train_y.shape)
#print(test1_X.shape)
#print(test1_y.shape)
print(test2_X.shape)
print(test2_y.shape)

mean = np.mean(train_X)
std = np.std(train_X)

print(mean)
print(std)

train_X -= mean
train_X /= std



# In[8]:




(289, 256, 512, 1)
(289, 256, 512, 1)
(289, 256, 512, 1)
(289, 256, 512, 1)
(1676, 256, 512, 1)
(1676, 256, 512, 1)
(289, 256, 512, 1)
(289, 256, 512, 1)
67.86644
56.091488


In [0]:
res1 = test1_X - mean
res1 = res1 / std


In [0]:
res2 = test2_X - mean
res2 = res2 / std


In [0]:
test1_X -= mean
test1_X /= std

test2_X -= mean
test2_X /= std

In [0]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, horizontal_flip=True, 
                             width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,fill_mode='constant')

datagen.fit(train_X)
weight_decay = 0.0001

In [0]:
def conv_block(input_img, filters, stage, block,strides=(1,1), wt='he_normal'):
    
    
    
    """if stage==5:
      strides=(1,1) 
     """ 
    
    """if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:"""
    bn_axis = 1
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
   
    
    x = Conv2D(filters, (1, 1), padding='same',strides=strides,name=conv_name_base + '2a',init=wt)(input_img)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    
    x = Conv2D(filters,(3, 3), padding='same',name=conv_name_base + '2b',init=wt)(x)
    x = BatchNormalization( name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    
    x = Conv2D(filters,(1,1), name=conv_name_base + '2c',init=wt)(x)
    x = BatchNormalization( name=bn_name_base + '2c')(x)
    
    
    shortcut = Conv2D(filters,(1, 1), strides=strides, 
                      name=conv_name_base + '1')(input_img)
   
    shortcut = BatchNormalization( name=bn_name_base + '1')(shortcut)

    x = layers.add([x,shortcut])
    x = Activation('relu')(x)
    
    return x

#model_l1

def ResNet50(learn = 3e-4, wt='he_normal',act='relu'):

    inputs = Input((256, 512,1)) 
    conv1 = Convolution2D(16, 3, 3, activation=act, border_mode='same',init=wt)(inputs) 
    conv1 = BatchNormalization()(conv1) 
    conv1 = Convolution2D(16, 3, 3, activation=act, border_mode='same',init=wt)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = conv_block(conv1,16, stage=1, block='a1')
    conv1 = conv_block(conv1,16, stage=1, block='a2')
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) 

    conv2 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(pool1)
    conv2 = BatchNormalization()(conv2) 
    conv2 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(conv2)    
    conv2 = BatchNormalization()(conv2) 
    conv2 = conv_block(conv2,32, stage=1, block='b1')
    conv2 = conv_block(conv2,32, stage=1, block='b2')
    #conv2 = conv_block(conv2,32, stage=1, block='b3')
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) 

    conv3 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(pool2)
    conv3 = BatchNormalization()(conv3) 
    conv3 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(conv3) 
    conv3 = BatchNormalization()(conv3) 
    conv3 = conv_block(conv3,64, stage=1, block='c1')
    conv3 = conv_block(conv3,64, stage=1, block='c2')
    #conv3 = conv_block(conv3,64, stage=1, block='c3')
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3) 

    conv4 = Convolution2D(128, 3, 3, activation=act, border_mode='same',init=wt)(pool3) 
    conv4 = BatchNormalization()(conv4) 
    conv4 = Convolution2D(128, 3, 3, activation=act, border_mode='same',init=wt)(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = conv_block(conv4,128, stage=1, block='d1')
    conv4 = conv_block(conv4,128, stage=1, block='d2')
    #conv4 = conv_block(conv4,128, stage=1, block='d3')
    #conv4 = Convolution2D(128, 3, 3, activation=act, border_mode='same',init=wt,dilation_rate=(8,8))(conv4)
    #conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4) 

    conv5 = Convolution2D(256, 3, 3, activation='relu', border_mode='same',init='he_normal')(pool4)
    conv5 = BatchNormalization()(conv5) 
    conv5 = Convolution2D(256, 3, 3, activation='relu', border_mode='same',init='he_normal')(conv5) 
    conv5 = BatchNormalization()(conv5)
    conv5 = conv_block(conv5,256, stage=1, block='e1')
    conv5 = conv_block(conv5,256, stage=1, block='e2')
    # conv5 = conv_block(conv5,256, stage=1, block='e3')
   

    input7 = UpSampling2D(size=(2, 2))(conv5) 
    input7 = Convolution2D(128,2,2,border_mode='same')(input7) 
    
    up7 = add([input7, conv4]) 
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same',init='he_normal')(up7) 
    conv7 = BatchNormalization()(conv7) 
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same',init='he_normal')(conv7) 
    conv7 = BatchNormalization()(conv7) 
    conv7 = conv_block(conv7,128, stage=1, block='f1')
    conv7 = conv_block(conv7,128, stage=1, block='f2')
    #conv7 = conv_block(conv7,128, stage=1, block='f3')

    input8 = UpSampling2D(size=(2, 2))(conv7) 
    input8 = Convolution2D(64,2,2,border_mode='same')(input8) 
    #up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4), conv3], axis=3)

    up8 = add([input8, conv3]) 
    conv8 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(up8) 
    conv8 = BatchNormalization()(conv8) 
    conv8 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(conv8) 
    conv8 = BatchNormalization()(conv8)
    conv8 = conv_block(conv8,64, stage=1, block='g1')
    conv8 = conv_block(conv8,64, stage=1, block='g2')
    #conv8 = conv_block(conv8,64, stage=1, block='g3')

    input9 = UpSampling2D(size=(2, 2))(conv8) 
    input9 = Convolution2D(32,2,2,border_mode='same')(input9) 
    #up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv2], axis=3)

    up9 = add([input9, conv2]) 
    conv9 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(up9) 
    conv9 = BatchNormalization()(conv9) 
    conv9 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(conv9) 
    conv9 = BatchNormalization()(conv9) 
    conv9 = conv_block(conv9,32, stage=1, block='h1')
    conv9 = conv_block(conv9,32, stage=1, block='h2')
    #conv9 = conv_block(conv9,32, stage=1, block='h3')

    input10 = UpSampling2D(size=(2, 2))(conv9) 
    input10 = Convolution2D(16,2,2,border_mode='same')(input10) 
    #up10 = concatenate([Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv9), conv1], axis=3)

    up10 = add([input10, conv1]) 
    conv10 = Convolution2D(16, 3, 3, activation=act, border_mode='same',init=wt)(up10)
    conv10 = BatchNormalization()(conv10) 
    conv10 = Convolution2D(16, 3, 3, activation=act, border_mode='same',init=wt)(conv10) 
    conv10 = BatchNormalization()(conv10) 
    conv10 = conv_block(conv10,16, stage=1, block='i1')
    conv10 = conv_block(conv10,16, stage=1, block='i2')
    #conv10 = conv_block(conv10,16, stage=1, block='i3')

    #conv11 = Dropout(0.5)(conv10)
    conv12 = Convolution2D(1, 1, 1,border_mode='same')(conv10) 

    conv13 = Activation('sigmoid')(conv12) 

    model = Model(input=inputs, output=conv13) 

    model.compile(Adam(lr=learn), loss='binary_crossentropy', metrics=[dice_coef,'accuracy']) 

    return model




In [31]:
K.clear_session()
model = ResNet50()

model.summary()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), padding="same", strides=(1, 1), name="res1a1_branch2a", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", name="res1a1_branch2b", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `C

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 512, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 512, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 512, 16) 2320        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (1, 1), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:151: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


In [0]:
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger

mc = ModelCheckpoint('model_loss', monitor='val_loss', save_best_only=True)
#tb = TensorBoard(write_images=True)
cv = CSVLogger('/content/drive/My Drive/Colab/results/latest/smriti_resnet_test1_l3.csv',append=True)



In [34]:
from google.colab import drive
drive.mount('/content/drive')




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [35]:
os.listdir('/content/drive/My Drive/Colab/results/best/')

['model_resnet_test1_l1.h5',
 'model_resnet_test1_l2.h5',
 'model_resnet_test1_weights_l2.h5',
 'dc_val_t1.ods',
 'dc_val_t1.gsheet',
 'model_resnet_test2.h5',
 'final.h5',
 'latest.ipynb',
 'best_weights.h5',
 'Copy of Best_78.ipynb',
 'Best_78.ipynb']

In [0]:
epochs =160
bestdc=0
total=0
for i in range(epochs):
    print(i)
    history= model.fit_generator(datagen.flow(train_X, train_y, batch_size=4), nb_epoch=1, samples_per_epoch=len(train_X), callbacks=[cv,mc])                           
    #history= model.fit(train_X, train_y, batch_size=4, nb_epoch=1, validation_data=(valid_X,valid_y), callbacks=[cv,mc])
    total=0
    for j in range(8):
        val_pred = model.predict(val_x[j], batch_size=4)
        val_result = np.zeros(val_pred.shape)
            #print test1_result.shape
        val_result[val_pred>0.5] = 1
            #np.save('patients/patient_' + 'str(j)' + '_' + 'str(i)' + '_.npy',val_result)
            #np.save('patients_sfu_dilaconv+dropout(0.2)/patient_' + str(j) + '_' + str(i) + '_.npy',val_result)
        dice = dc(val_result,val_gt[j])
        # print("Volume "+str(j)+": "+str(dice))
        total+= dice
    mean = total/8.0
    print("Average dc: "+str(mean))
    if mean>bestdc:
        bestdc=mean
        model.save_weights('/content/drive/My Drive/Colab/results/latest/model_resnet_test1_weights_l3.h5') 
    print("Best dc: "+str(bestdc))


0
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., steps_per_epoch=419, epochs=1)`
  


Epoch 1/1
419/419 [==============================] - 405s 966ms/step - loss: 0.1835 - dice_coef: 0.0259 - acc: 0.9544


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


NameError: ignored

In [0]:
model.load_weights('/content/drive/My Drive/Colab/results/best/final.h5')

In [0]:
#TEST1
dcoefarr=[]


  
val_pred = model.predict(test1_X, batch_size=4)
score = model.evaluate(test1_X, test1_y)
print(score)
val_result = np.zeros(val_pred.shape)                                                                                                                                                        
val_result[val_pred>0.5] = 1
dcoef = dc(val_result,test1_y)
prec  = precision(val_result,test1_y)
rec   = recall(val_result,test1_y)
dcoefarr.append(dcoef)
print ("Dice: ",dcoef)
print("Precision: ",prec)
print("recall: ",rec)

In [38]:
#TEST2
dcoefarr=[]


  
val_pred = model.predict(test2_X, batch_size=4)
score = model.evaluate(test2_X, test2_y)
print(score)
val_result = np.zeros(val_pred.shape)                                                                                                                                                        
val_result[val_pred>0.5] = 1
dcoef = dc(val_result,test2_y)
prec  = precision(val_result,test2_y)
rec   = recall(val_result,test2_y)
dcoefarr.append(dcoef)
print ("Dice: ",dcoef)
print("Precision: ",prec)
print("recall: ",rec)

289/289 [==============================] - 11s 40ms/step
[0.008661655480848547, 0.4280723438640109, 0.9966605691348805]
Dice:  0.8345107643358766
Precision:  0.7724201399820784
recall:  0.9074561977545481


In [0]:
### TEST1_PREDICTION

In [0]:
#cirrus 1 nd 2 ,nidek 3 nd 4,spectralis 5 nd 6,topcon 7 nd 8

def test1results(model):
    
    frames = [(0,128),(128,256),(256,261),(261,266),(266,315),(315,364),(364,492),(492,620)]
    test1resarray = []
    
    
    for item in frames:
        x = item[0]
        y = item[1]
        
        test1_pred = model.predict(res1[x:y], batch_size=4)
        test1_result = np.zeros(test1_pred.shape)
        #print test1_result.shape
        test1_result[test1_pred>0.5] = 1
        test1resarray.append(test1_result)
        
        a = test1_result
        b = test1_y1[x:y]
        c = test1_y2[x:y]
        d = test1_y3[x:y]
        
        print("")
        print(dc(a, b))
        print(dc(a, c))
        print(dc(a, d))

        print(recall(a, b))
        print(recall(a, c))
        print(recall(a, d))

        print(precision(a, b))
        print(precision(a, c))
        print(precision(a, d))
        
    '''  
    for i in xrange(test1_pred.shape[0]):
            imsave('test1_results/'+str(i+x)+'.jpg',test1_X[i+x,:,:,0])
            imsave('test1_results/'+str(i+x)+'g.jpg',test1_y3[i+x,:,:,0])
            imsave('test1_results/'+str(i+x)+'p.jpg',test1_result[i,:,:,0]*255)
    
    test1resarray = np.array(test1resarray)
    np.save('test1results')
    '''

In [40]:
test1results(model)
#cirrus 1 nd 2 ,nidek 3 nd 4,spectralis 5 nd 6,topcon 7 nd 8


0.13314641198633137
0.14182358450954427
0.14496492153758966
0.07776867963152508
0.08175842999981381
0.08634416612347519
0.46244674376141204
0.5345100426049909
0.45149117468046257

0.7345194751665866
0.7284667571234735
0.7590964012052063
0.631222253707377
0.6204840163650233
0.7101241916099125
0.8782402996353122
0.8819528862897131
0.815323455005421

0.5113581127061797
0.5052846914422093
0.5240407632856849
0.37560696943730365
0.36372155287817937
0.3951725205013152
0.8007713386785751
0.8272607327717446
0.7776311783213234

0.8488289136143536
0.8288038828306084
0.8047543291215152
0.8986886088968886
0.9025274427429191
0.9561405244928254
0.8042108896367245
0.7662150890211983
0.6947536456985072

0.6748782662368836
0.68142180994616
0.7169126863580729
0.5322702784569117
0.5339525196063846
0.5910472439030321
0.9218691448699693
0.9414299632766244
0.9108896050363486

0.7139737991266376
0.763676963155936
0.7678571428571429
0.592749244712991
0.64366373902133
0.6900072939460248
0.8975297346752058
0.93

In [0]:
##TEST2_PREDICTION

In [0]:
#cirrus1 nd 2 ,nidek 3 ,spectralis 4 nd 5,topcon 6 nd 7
def test2results(model):
    
    frames = [(0,128),(128,256),(256,261),(261,268),(268,275),(275,282),(282,289)]
    test2resarray = []
    
    print("\nTEST 2 RESULTS: ")
    for item in frames:
        x = item[0]
        y = item[1]
        
        test2_pred = model.predict(res2[x:y], batch_size=4)
        test2_result = np.zeros(test2_pred.shape)
        #print test1_result.shape
        test2_result[test2_pred>0.5] = 1
        test2resarray.append(test2_result)
        
        a = test2_result
        b = test2_y1[x:y]
        c = test2_y2[x:y]
        d = test2_y3[x:y]
        
        print("")
        print(dc(a, b))
        print(dc(a, c))
        print(dc(a, d))

        print(recall(a, b))
        print(recall(a, c))
        print(recall(a, d))

        print(precision(a, b))
        print(precision(a, c))
        print(precision(a, d))
        #for i in xrange(test2_pred.shape[0]):
        #    imsave('test2_results/'+str(i+x)+'.jpg',test2_X[i+x,:,:,0])
        #   imsave('test2_results/'+str(i+x)+'g.jpg',test2_y3[i+x,:,:,0])
        #  imsave('test2_results/'+str(i+x)+'p.jpg',test2_result[i,:,:,0]*255)
    
    #test2resarray = np.array(test2resarray)
    #np.save('test2results')

In [42]:
test2results(model)
#cirrus 1 nd 2 ,nidek 3 ,spectralis 4 nd 5,topcon 6 nd 7


TEST 2 RESULTS: 

0.7690560934848891
0.7917641204279672
0.7762575894942169
0.7756562017498714
0.7789146451912956
0.8492380752006252
0.7625673590204164
0.805044653021985
0.7148278391985226

0.8635419859404199
0.8797775163530773
0.8432443072332843
0.9328800995476563
0.9218316746826382
0.9644700125009628
0.803798138203638
0.8413929881233405
0.7490900389753218

0.8548865167374857
0.836294253004988
0.81232398934587
0.8691852705227813
0.8815385078944197
0.9260148596683545
0.841050599526904
0.795467522227358
0.7234971043258381

0.7262848589011477
0.7070459331929301
0.7442040415463825
0.6075221444140426
0.5765652173913044
0.6549502357255107
0.9027634208531459
0.9138584522086692
0.8616222176280064

0.8359430207907167
0.8629282311124555
0.8875965105785621
0.7395123877208568
0.7733048693707432
0.8355988521371394
0.9612933578546683
0.9760489286172533
0.946495017321757

0.7884738279207576
0.7858307819966237
0.8058643564656347
0.6935813573180704
0.6834901199542084
0.7326907188102451
0.9134473011172

In [0]:
for i in range(0,287):
    imsave('/content/drive/My Drive/Colab/images/'+str('i')+'.jpg',test1_X[i,:,:,0])
    imsave('/content/drive/My Drive/Colab/images/'+str('i1')+'.jpg',test1_y1[i,:,:,0])
    imsave('/content/drive/My Drive/Colab/images/'+str('i2')+'.jpg',test1_y2[i,:,:,0])
    imsave('/content/drive/My Drive/Colab/images/'+str('i3')+'.jpg',test1_y3[i,:,:,0])
    #imsave('/content/drive/My Drive/results/smriti_resnet_with_identity/G1/test1/'+str(i+1)+'a.jpg',test_y[i,:,:,0])
    #imsave('/content/drive/My Drive/Colab/results/G2/test2/'+str(i+1)+'b.jpg',val_result[i,:,:,0]*255)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  del sys.path[0]
